# Carga de Datos desde Volúmenes en Databricks

## ¿Qué son los Volúmenes en Databricks?

Los **volúmenes** en Databricks son una funcionalidad de Unity Catalog que proporciona almacenamiento unificado y gobernado para archivos no estructurados. En este ejemplo, utilizamos un volumen para cargar múltiples archivos CSV de datos olímpicos.

### Estructura de la Ruta del Volumen

La ruta `/Volumes/workspace/dbtest/dataclase2/` sigue la estructura estándar de volúmenes:

- **`/Volumes/`**: Prefijo que indica el uso del sistema de volúmenes
- **`workspace`**: Nombre del catálogo (catalog)
- **`dbtest`**: Nombre del esquema (schema) 
- **`dataclase2`**: Nombre del volumen específico

### Dataset Olímpico

Trabajaremos con un conjunto de datos relacionados con los Juegos Olímpicos que incluye:

- **`Athletes.csv`**: Información de los atletas participantes
- **`Coaches.csv`**: Datos de los entrenadores
- **`EntriesGender.csv`**: Distribución por género de las participaciones
- **`Medals.csv`**: Registro de medallas otorgadas
- **`Teams.csv`**: Información de los equipos nacionales

### Ventajas de usar Volúmenes para este caso:

- **Organización**: Todos los archivos relacionados en una ubicación centralizada
- **Gobernanza**: Control de acceso unificado para todo el dataset
- **Rendimiento**: Lectura optimizada de múltiples archivos CSV
- **Colaboración**: Acceso compartido y seguro entre equipos

---

El siguiente código demuestra cómo cargar estos archivos CSV desde el volumen utilizando Spark:

In [0]:
# Definimos la ruta base del volumen
base_path = "/Volumes/workspace/dbtest/dataclase2/"

# Leemos los archivos CSV como DataFrames
athletes_df = spark.read.option("header", True).option("delimiter", ",").csv(f"{base_path}Athletes.csv")
coaches_df = spark.read.option("header", True).option("delimiter", ",").csv(f"{base_path}Coaches.csv")
entries_gender_df = spark.read.option("header", True).option("delimiter", ",").csv(f"{base_path}EntriesGender.csv")
medals_df = spark.read.option("header", True).option("delimiter", ",").csv(f"{base_path}Medals.csv")
teams_df = spark.read.option("header", True).option("delimiter", ",").csv(f"{base_path}Teams.csv")

# Exploración de DataFrames de Spark

## Verificación del Esquema y Contenido

Una vez cargados los archivos CSV como DataFrames de Spark, es fundamental realizar una exploración inicial para entender la estructura y el contenido de los datos. Para esto utilizamos dos métodos esenciales:

### `printSchema()` - Análisis de la Estructura

El método **`printSchema()`** nos permite visualizar la estructura completa del DataFrame, mostrando:

- **Nombres de las columnas**: Identificación de cada campo disponible
- **Tipos de datos**: String, Integer, Double, Boolean, etc.
- **Nullabilidad**: Si la columna permite valores nulos o no
- **Jerarquía**: Estructura en forma de árbol que facilita la lectura

Esta información es crucial para:
- Validar que la carga de datos fue correcta
- Identificar posibles problemas de tipos de datos
- Planificar transformaciones futuras
- Entender la calidad de los datos cargados

### `show()` - Previsualización del Contenido

El método **`show(n)`** muestra las primeras **n** filas del DataFrame de manera tabular:

- **Muestreo rápido**: Vista de los datos reales contenidos
- **Validación de contenido**: Verificación de que los datos son consistentes
- **Detección de patrones**: Identificación de formatos, valores nulos, etc.
- **Control de calidad**: Revisión inicial antes del procesamiento

En nuestro caso, `show(5)` muestra las primeras 5 filas de cada DataFrame, proporcionando una muestra representativa sin sobrecargar la salida.

### Estrategia de Exploración

Para el dataset olímpico, examinaremos únicamente el DataFrame de **atletas** (`athletes_df`) como muestra inicial, manteniendo comentados los demás para evitar una salida excesiva. Esta aproximación nos permite:

- Entender la estructura general del dataset
- Validar la correcta carga de datos
- Identificar patrones antes de procesar todos los DataFrames

---

El siguiente código ejecuta la exploración del esquema y muestra una preview del DataFrame de atletas:

In [0]:
# mostramos el esquema de cada DataFrame
# athletes_df.printSchema()
coaches_df.printSchema()
# entries_gender_df.printSchema()
# medals_df.printSchema()
# teams_df.printSchema()

# Mostramos un preview de cada DataFrame
# athletes_df.show(5)
# coaches_df.show(5)
# entries_gender_df.show(5)
medals_df.show(5)
# teams_df.show(5)

# Transformación de Tipos de Datos en Spark

## Conversión de Columnas Numéricas

Después de cargar los datos CSV, es común encontrar que las columnas numéricas se interpreten como tipo `String` por defecto. Para realizar operaciones matemáticas y análisis estadístico, necesitamos convertir estas columnas a sus tipos de datos apropiados.

### Importación de Funciones de PySpark

Importamos las herramientas necesarias para la transformación:

- **`functions as F`**: Módulo que contiene funciones de columna para transformaciones
- **`IntegerType`**: Tipo de dato específico para números enteros
- **`F.col()`**: Función que referencia una columna existente del DataFrame

### Método `withColumn()` - Transformación de Columnas

El método **`withColumn()`** permite:

- **Modificar columnas existentes**: Cambiar el tipo de dato manteniendo el mismo nombre
- **Crear nuevas columnas**: Agregar columnas calculadas al DataFrame
- **Transformaciones en cadena**: Aplicar múltiples transformaciones de forma secuencial

### Conversión con `cast()`

La función **`cast()`** realiza la conversión de tipos de datos:

- **Conversión segura**: Maneja automáticamente valores que no pueden convertirse
- **Validación implícita**: Los valores no numéricos se convierten en `null`
- **Optimización**: Mejora el rendimiento para operaciones matemáticas posteriores

### DataFrame Target: `entries_gender_df`

Para el DataFrame de distribución por género, convertimos las columnas numéricas:

- **`Female`**: Número de participantes femeninas (String → Integer)
- **`Male`**: Número de participantes masculinos (String → Integer) 
- **`Total`**: Total de participantes (String → Integer)

Esta transformación es esencial para poder realizar:
- Cálculos estadísticos
- Agregaciones numéricas
- Visualizaciones de datos
- Análisis comparativo entre géneros

---

El siguiente código aplica la transformación de tipos de datos y verifica el resultado:

In [0]:
# Importamos funciones de PySpark
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

# Convertimos las columnas a IntegerType usando F.col
entries_gender_df = entries_gender_df \
    .withColumn("Female", F.col("Female").cast(IntegerType())) \
    .withColumn("Male", F.col("Male").cast(IntegerType())) \
    .withColumn("Total", F.col("Total").cast(IntegerType()))

# Mostramos un preview para verificar
entries_gender_df.show(5)

In [0]:
entries_gender_df.printSchema()

# Análisis y Ranking de Datos en Spark

## Creación de Rankings con Ordenamiento y Selección

Una de las tareas más comunes en análisis de datos deportivos es identificar los países con mejor desempeño. En este caso, crearemos un ranking de países basado en el número de medallas de oro obtenidas.

### Método `orderBy()` - Ordenamiento de Datos

La función **`orderBy()`** permite organizar las filas del DataFrame según criterios específicos:

- **Ordenamiento ascendente**: Por defecto, ordena de menor a mayor
- **Ordenamiento descendente**: Usando `.desc()` ordena de mayor a menor
- **Múltiples columnas**: Permite ordenar por varias columnas simultáneamente
- **Referencia de columnas**: Utiliza `F.col("nombre_columna")` para referenciar columnas

En nuestro caso, `F.col("Gold").desc()` ordena por medallas de oro de mayor a menor, colocando a los países con más medallas al inicio.

### Método `select()` - Selección de Columnas

La función **`select()`** permite especificar qué columnas queremos mantener en el resultado:

- **Reducción de datos**: Elimina columnas innecesarias para el análisis específico
- **Optimización de rendimiento**: Procesa solo los datos requeridos
- **Claridad en resultados**: Presenta únicamente la información relevante
- **Preparación para visualización**: Estructura los datos para reportes o gráficos

Seleccionamos:
- **`TeamCountry`**: Nombre del país/equipo
- **`Gold`**: Cantidad de medallas de oro obtenidas

### Encadenamiento de Operaciones

PySpark permite **encadenar métodos** usando el operador `\` para mejorar la legibilidad:

- **Flujo secuencial**: Las operaciones se aplican una tras otra
- **Código limpio**: Evita variables temporales innecesarias
- **Legibilidad**: Cada operación en una línea separada
- **Mantenibilidad**: Fácil de modificar o extender

### Objetivo del Análisis

Esta consulta nos permite identificar:

- **Top performers**: Países con mayor éxito en medallas de oro
- **Distribución del rendimiento**: Cómo se distribuye el éxito entre países
- **Análisis comparativo**: Diferencias entre el desempeño de diferentes naciones
- **Base para visualizaciones**: Datos preparados para gráficos de ranking

---

El siguiente código crea el ranking de países por medallas de oro y muestra los 10 primeros resultados:

In [0]:
from pyspark.sql import functions as F

# Ordenamos por la columna "Gold" de manera descendente y seleccionamos país y medallas de oro
top_gold_medal_countries = medals_df \
    .orderBy(F.col("Gold").desc()) \
    .select(F.col("TeamCountry"), F.col("Gold"))

# Mostramos los primeros resultados
top_gold_medal_countries.show(10)

# Creación de Columnas Calculadas en Spark

## Análisis de Proporciones y Promedios por Género

Para entender mejor la distribución de género en las diferentes disciplinas olímpicas, calculamos las proporciones relativas de participación femenina y masculina. Esto nos permite realizar comparaciones equitativas independientemente del tamaño total de cada disciplina.

### Importación de Funciones

Reutilizamos el módulo **`functions as F`** que ya importamos anteriormente, aprovechando sus capacidades para:

- **Operaciones matemáticas**: Realizar cálculos entre columnas
- **Referencia de columnas**: Usar `F.col()` para operaciones aritméticas
- **Transformaciones**: Crear nuevas columnas basadas en datos existentes

### Método `withColumn()` para Cálculos

Utilizamos **`withColumn()`** para crear columnas calculadas que representen proporciones:

- **Cálculos en tiempo real**: Las operaciones se realizan durante la ejecución
- **Preservación de datos**: Mantiene todas las columnas originales
- **Múltiples transformaciones**: Permite encadenar varios cálculos

### Operaciones Aritméticas entre Columnas

Las operaciones matemáticas en Spark se realizan usando `F.col()`:

- **División**: `F.col('Female') / F.col('Total')` calcula la proporción femenina
- **Precisión decimal**: Automáticamente maneja la precisión de los cálculos
- **Manejo de nulos**: Gestiona valores nulos de manera segura
- **Optimización**: Las operaciones se distribuyen eficientemente

### Columnas Calculadas Específicas

Creamos dos nuevas columnas de proporción:

- **`Avg_Female`**: Proporción de participantes femeninas (Female/Total)
  - Valor entre 0 y 1, donde 1 = 100% participación femenina
- **`Avg_Male`**: Proporción de participantes masculinos (Male/Total)
  - Valor entre 0 y 1, donde 1 = 100% participación masculina

### Valor Analítico

Este análisis proporcional nos permite:

- **Comparaciones equitativas**: Evaluar representación de género independientemente del tamaño
- **Identificación de patrones**: Detectar disciplinas con mayor/menor equilibrio de género
- **Análisis de diversidad**: Medir la inclusión en diferentes deportes olímpicos
- **Base para métricas**: Calcular estadísticas de paridad de género

---

El siguiente código calcula las proporciones de género por disciplina y muestra los primeros 10 resultados:

In [0]:
from pyspark.sql import functions as F

# Calculamos el promedio de entradas por género para cada disciplina
average_entries_by_gender = entries_gender_df \
    .withColumn('Avg_Female', F.col('Female') / F.col('Total')) \
    .withColumn('Avg_Male', F.col('Male') / F.col('Total'))

# Mostramos los resultados
average_entries_by_gender.show(10)

In [0]:
medals_df.display()

medals_df = medals_df.withColumn("Gold", F.col("Gold").cast("int"))

medals_df.display()

df_oro = (
    medals_df
    .filter(F.col("Gold") > 10)
    .select(F.col("TeamCountry"), F.col("Gold"))
    .orderBy(F.col("Gold").desc())
)
df_oro.display()